In [1]:
import pandas as pd
import numpy as np
import re

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)  # me muestre todas las columnas
#pd.set_option('display.max_rows', None)  # me muestre todas las filas

# pa pintar
import pylab as plt   # import matplotlib.pyplot as plt
import seaborn as sns

# para que salga el grafico
# %matplotlib inline

In [2]:
df_origin = pd.read_csv(r'..\data\attacks.csv', encoding='latin1')
df = df_origin.copy()

In [3]:
df.head(10)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN
5,2018.06.03.b,03-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,"Flat Rock, Ballina",Kite surfing,Chris,M,NaN,"No injury, board bitten",N,NaN,NaN,"Daily Telegraph, 6/4/2018",2018.06.03.b-FlatRock.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.03.b,2018.06.03.b,6298.0,NaN,NaN
6,2018.06.03.a,03-Jun-2018,2018.0,Unprovoked,BRAZIL,Pernambuco,"Piedade Beach, Recife",Swimming,Jose Ernesto da Silva,M,18,FATAL,Y,Late afternoon,Tiger shark,"Diario de Pernambuco, 6/4/2018",2018.06.03.a-daSilva.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.03.a,2018.06.03.a,6297.0,NaN,NaN
7,2018.05.27,27-May-2018,2018.0,Unprovoked,USA,Florida,"Lighhouse Point Park, Ponce Inlet, Volusia County",Fishing,male,M,52,Minor injury to foot. PROVOKED INCIDENT,N,NaN,"Lemon shark, 3'","K. McMurray, TrackingSharks.com",2018.05.27-Ponce.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.05.27,2018.05.27,6296.0,NaN,NaN
8,2018.05.26.b,26-May-2018,2018.0,Unprovoked,USA,Florida,"Cocoa Beach, Brevard County",Walking,Cody High,M,15,Lower left leg bitten,N,17h00,"Bull shark, 6'","K.McMurray, TrackingSharks.com",2018.05.26.b-High.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.05.26.b,2018.05.26.b,6295.0,NaN,NaN
9,2018.05.26.a,26-May-2018,2018.0,Unprovoked,USA,Florida,"Daytona Beach, Volusia County",Standing,male,M,12,Minor injury to foot,N,14h00,NaN,"K. McMurray, Tracking Sharks.com",2018.05.26.a-DaytonaBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.05.26.a,2018.05.26.a,6294.0,NaN,NaN


In [4]:
df.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25723 entries, 0 to 25722
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case Number             8702 non-null   object 
 1   Date                    6302 non-null   object 
 2   Year                    6300 non-null   float64
 3   Type                    6298 non-null   object 
 4   Country                 6252 non-null   object 
 5   Area                    5847 non-null   object 
 6   Location                5762 non-null   object 
 7   Activity                5758 non-null   object 
 8   Name                    6092 non-null   object 
 9   Sex                     5737 non-null   object 
 10  Age                     3471 non-null   object 
 11  Injury                  6274 non-null   object 
 12  Fatal (Y/N)             5763 non-null   object 
 13  Time                    2948 non-null   object 
 14  Species                 3464 non-null 

In [6]:
df.shape

(25723, 24)

In [7]:
nan_cols = df.isna().sum()

nan_cols[nan_cols>0]

Case Number               17021
Date                      19421
Year                      19423
Type                      19425
Country                   19471
Area                      19876
Location                  19961
Activity                  19965
Name                      19631
Sex                       19986
Age                       22252
Injury                    19449
Fatal (Y/N)               19960
Time                      22775
Species                   22259
Investigator or Source    19438
pdf                       19421
href formula              19422
href                      19421
Case Number.1             19421
Case Number.2             19421
original order            19414
Unnamed: 22               25722
Unnamed: 23               25721
dtype: int64

In [8]:
df.duplicated().any()

True

In [9]:
df = df.drop_duplicates()

In [10]:
df.shape, df_origin.shape

((6312, 24), (25723, 24))

In [11]:
df.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


### Número de nulos por cada columna

In [12]:
nan_cols = df.isna().sum()

nan_cols[nan_cols>0]

Case Number                  2
Date                        10
Year                        12
Type                        14
Country                     60
Area                       465
Location                   550
Activity                   554
Name                       220
Sex                        575
Age                       2841
Injury                      38
Fatal (Y/N)                549
Time                      3364
Species                   2848
Investigator or Source      27
pdf                         10
href formula                11
href                        10
Case Number.1               10
Case Number.2               10
original order               3
Unnamed: 22               6311
Unnamed: 23               6310
dtype: int64

In [13]:
df.reset_index(drop=True, inplace=True)

In [14]:
nan_rows = df.isna().sum(axis=1)

# Filtrar las filas con valores nulos
rows_with_nulls = nan_rows[nan_rows > 0]

rows_with_nulls.tail()

6307    22
6308    22
6309    23
6310    24
6311    23
dtype: int64

# 1. Limpieza de las columnas (Case Number, Date,	Year, Case Number.1,	Case Number.2,	original order,	Unnamed: 22,	Unnamed: 23	)

### Posición de los valores nulos en 'Date'

In [15]:
bad_index = df['Date'][df['Date'].isna()].index

bad_index

Int64Index([6302, 6303, 6304, 6305, 6306, 6307, 6308, 6309, 6310, 6311], dtype='int64')

### Obtener filas por índice o etiqueta de fila

In [16]:
df.loc[[6302, 6303, 6304, 6305, 6306, 6307, 6308, 6309, 6310, 6311]]

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
6302,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6304.0,NaN,NaN
6303,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6305.0,NaN,NaN
6304,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6306.0,NaN,NaN
6305,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6307.0,NaN,NaN
6306,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6308.0,NaN,NaN
6307,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6309.0,NaN,NaN
6308,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6310.0,NaN,NaN
6309,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6311,xx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### No hay ninguna, ya que los valores en las columnas 'Case Number' cuando 'Date' es nulo también so nulos

### Relleno los valores nulos de la columna 'Date' 

In [17]:
df.drop(index=df[df['Date'].isna()].index, inplace=True)
df.shape

(6302, 24)

In [18]:
def clean_date(date):
    pattern = r'(\d{2})[/-](\w{3})[/-](\d{4})'  # Patrón regex para identificar la fecha
    match = re.match(pattern, date)  # Buscar el patrón en la fecha

    if match:
        day = match.group(1)  # Extraer el día
        month = match.group(2)  # Extraer el mes
        year = match.group(3)  # Extraer el año

        # Convertir el mes a formato deseado (en este caso, triple abreviatura)
        month_dict = {
            'Jan': 'Jan', 'Feb': 'Feb', 'Mar': 'Mar', 'Apr': 'Apr', 'May': 'May', 'Jun': 'Jun',
            'Jul': 'Jul', 'Aug': 'Aug', 'Sep': 'Sep', 'Oct': 'Oct', 'Nov': 'Nov', 'Dec': 'Dec'
        }

        if month in month_dict:
            month = month_dict[month]

        # Devolver la fecha en el formato deseado
        return f'{day}-{month}-{year}'

    else:
        return None  # Devolver None si no se encuentra el patrón en la fecha

In [19]:
df['Date'] = df['Date'].apply(clean_date)

df['Date'].head()

0    25-Jun-2018
1    18-Jun-2018
2    09-Jun-2018
3    08-Jun-2018
4    04-Jun-2018
Name: Date, dtype: object

In [20]:
df['Date'].isna().sum()

1549

### Cambio el tipo de dato de la columna 'Date' a datetime

In [21]:
df['Date'] = pd.to_datetime(df['Date'], format='%d-%b-%Y')

df['Date'].head()

0   2018-06-25
1   2018-06-18
2   2018-06-09
3   2018-06-08
4   2018-06-04
Name: Date, dtype: datetime64[ns]

## 1.1 Funcion para limpiar las columnas de 'Case Numbers_' 

In [22]:
# limpio la columna con una funcion regex



def clean_date_case_number(case_number):
    case_number = str(case_number)
    pattern = r'(\d{4})\.(\d{2})\.(\d{2})'
    match = re.search(pattern, case_number)

    if match:
        year = int(match.group(1))
        month = int(match.group(2))
        day = int(match.group(3))

        if month == 0 or day == 0:
            return np.nan

        try:
            return pd.to_datetime(f'{year}-{month:02d}-{day:02d}')
        except ValueError:
            return np.nan
    else:
        return np.nan

In [23]:
df['Case Number'] = df['Case Number'].apply(clean_date_case_number)
df['Case Number'].head()

0   2018-06-25
1   2018-06-18
2   2018-06-09
3   2018-06-08
4   2018-06-04
Name: Case Number, dtype: datetime64[ns]

### Paso el tipo de dato de la columna 'Case Number' a datetime

In [24]:
df['Case Number'] = pd.to_datetime(df['Case Number'], format='%Y-%m-%d')
df['Case Number']

0      2018-06-25
1      2018-06-18
2      2018-06-09
3      2018-06-08
4      2018-06-04
          ...    
6297          NaT
6298          NaT
6299          NaT
6300          NaT
6301          NaT
Name: Case Number, Length: 6302, dtype: datetime64[ns]

In [25]:
df['Case Number'].isna().sum()

880

### Relleno los valores nulos de 'Date' con los de 'Case Number'

In [26]:
df['Date'] = df['Date'].combine_first(df['Case Number'])

df['Date']

0      2018-06-25
1      2018-06-18
2      2018-06-09
3      2018-06-08
4      2018-06-04
          ...    
6297          NaT
6298          NaT
6299          NaT
6300          NaT
6301          NaT
Name: Date, Length: 6302, dtype: datetime64[ns]

### Valores nulos ahora despues de haber hecho el filtro

In [27]:
df['Date'].isna().sum()

875

In [28]:
df.drop(index=df[df['Date'].isna()].index, inplace=True)
df.shape

(5427, 24)

### Limpiar valores en 'Case Numbre.1'

In [30]:
df['Case Number.1'] = df['Case Number.1'].apply(clean_date_case_number)
df['Case Number.1'].head()

0   2018-06-25
1   2018-06-18
2   2018-06-09
3   2018-06-08
4   2018-06-04
Name: Case Number.1, dtype: datetime64[ns]

### Paso el tipo de dato de la columna 'Case Number.1' a datetime

In [31]:
df['Case Number.1'] = pd.to_datetime(df['Case Number.1'], format='%Y-%m-%d')
df['Case Number.1']

0      2018-06-25
1      2018-06-18
2      2018-06-09
3      2018-06-08
4      2018-06-04
          ...    
6150   1753-10-27
6151   1751-07-27
6155   1742-12-17
6156   1738-04-06
6160   1703-03-26
Name: Case Number.1, Length: 5427, dtype: datetime64[ns]

### Relleno los valores nulos de 'Date' con los de 'Case Number.1'

In [32]:
df['Date'] = df['Date'].combine_first(df['Case Number.1'])

df['Date']

0      2018-06-25
1      2018-06-18
2      2018-06-09
3      2018-06-08
4      2018-06-04
          ...    
6150   1753-10-27
6151   1751-07-27
6155   1742-12-17
6156   1738-04-06
6160   1703-03-26
Name: Date, Length: 5427, dtype: datetime64[ns]

### Limpiar valores en 'Case Numbre.2'

In [33]:
df['Case Number.2'] = df['Case Number.2'].apply(clean_date_case_number)
df['Case Number.2'].head()

0   2018-06-25
1   2018-06-18
2   2018-06-09
3   2018-06-08
4   2018-06-04
Name: Case Number.2, dtype: datetime64[ns]

### Paso el tipo de dato de la columna 'Case Number.2' a datetime

In [34]:
df['Case Number.2'] = pd.to_datetime(df['Case Number.2'], format='%Y-%m-%d')
df['Case Number.2']

0      2018-06-25
1      2018-06-18
2      2018-06-09
3      2018-06-08
4      2018-06-04
          ...    
6150   1753-10-27
6151   1751-07-27
6155   1742-12-17
6156   1738-04-06
6160   1703-03-26
Name: Case Number.2, Length: 5427, dtype: datetime64[ns]

### Relleno los valores nulos de 'Date' con los de 'Case Number.2'


In [35]:
df['Date'] = df['Date'].combine_first(df['Case Number.2'])

df['Date']

0      2018-06-25
1      2018-06-18
2      2018-06-09
3      2018-06-08
4      2018-06-04
          ...    
6150   1753-10-27
6151   1751-07-27
6155   1742-12-17
6156   1738-04-06
6160   1703-03-26
Name: Date, Length: 5427, dtype: datetime64[ns]

In [36]:
df['Date'].isna().sum()

0

### Reinicio el indice con las fechas ( hacer este paso al final, cuando el df esté limpio del todo)

In [37]:
df_indate = df.set_index('Date')
df_indate.head(20)

,Case Number,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
Date,,,,,,,,,,,,,,,,,,,,,,,
2018-06-25,2018-06-25,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-25,2018-06-25,6303.0,NaN,NaN
2018-06-18,2018-06-18,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-18,2018-06-18,6302.0,NaN,NaN
2018-06-09,2018-06-09,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-09,2018-06-09,6301.0,NaN,NaN
2018-06-08,2018-06-08,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-08,2018-06-08,6300.0,NaN,NaN
2018-06-04,2018-06-04,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-04,2018-06-04,6299.0,NaN,NaN
2018-06-03,2018-06-03,2018.0,Unprovoked,AUSTRALIA,New South Wales,"Flat Rock, Ballina",Kite surfing,Chris,M,NaN,"No injury, board bitten",N,NaN,NaN,"Daily Telegraph, 6/4/2018",2018.06.03.b-FlatRock.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-03,2018-06-03,6298.0,NaN,NaN
2018-06-03,2018-06-03,2018.0,Unprovoked,BRAZIL,Pernambuco,"Piedade Beach, Recife",Swimming,Jose Ernesto da Silva,M,18,FATAL,Y,Late afternoon,Tiger shark,"Diario de Pernambuco, 6/4/2018",2018.06.03.a-daSilva.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-03,2018-06-03,6297.0,NaN,NaN
2018-05-27,2018-05-27,2018.0,Unprovoked,USA,Florida,"Lighhouse Point Park, Ponce Inlet, Volusia County",Fishing,male,M,52,Minor injury to foot. PROVOKED INCIDENT,N,NaN,"Lemon shark, 3'","K. McMurray, TrackingSharks.com",2018.05.27-Ponce.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-05-27,2018-05-27,6296.0,NaN,NaN
2018-05-26,2018-05-26,2018.0,Unprovoked,USA,Florida,"Cocoa Beach, Brevard County",Walking,Cody High,M,15,Lower left leg bitten,N,17h00,"Bull shark, 6'","K.McMurray, TrackingSharks.com",2018.05.26.b-High.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-05-26,2018-05-26,6295.0,NaN,NaN


# 2. Limpiar Type

In [38]:
df['Type'].unique()

array(['Boating', 'Unprovoked', 'Invalid', 'Provoked', 'Questionable',
       'Sea Disaster', nan, 'Boat', 'Boatomg'], dtype=object)

In [39]:
df['Type'] = df['Type'].str.replace('Boating', 'Boat')
df['Type'] = df['Type'].str.replace('Boatomg', 'Boat')
df['Type'].unique()

array(['Boat', 'Unprovoked', 'Invalid', 'Provoked', 'Questionable',
       'Sea Disaster', nan], dtype=object)

In [40]:
'''
'Boat': Se refiere a un ataque de tiburón que ocurrió mientras la víctima se encontraba en un bote o embarcación.

'Unprovoked': Representa un ataque de tiburón en el que la víctima fue atacada sin provocación aparente por parte de la persona.

'Invalid': Este valor se utiliza cuando la información disponible no es suficiente para determinar si el incidente 
fue un ataque de tiburón válido o si fue causado por otra causa.

'Provoked': Indica que el ataque ocurrió como respuesta a una provocación o interacción directa con el tiburón por parte 
de la persona, como tocarlo o molestarlo de alguna manera.

'Questionable': Se utiliza cuando la categorización del ataque de tiburón no está clara y se requiere una mayor investigación 
o información para determinar su naturaleza.

'Sea Disaster': Este valor se aplica cuando el incidente no fue un ataque de tiburón directo, sino más bien una situación en 
la que muchas personas estuvieron involucradas y / o personas murieron debido a un desastre marino, como un naufragio, sin 
ser ocasionado específicamente por un tiburón.
'''

"\n'Boat': Se refiere a un ataque de tiburón que ocurrió mientras la víctima se encontraba en un bote o embarcación.\n\n'Unprovoked': Representa un ataque de tiburón en el que la víctima fue atacada sin provocación aparente por parte de la persona.\n\n'Invalid': Este valor se utiliza cuando la información disponible no es suficiente para determinar si el incidente \nfue un ataque de tiburón válido o si fue causado por otra causa.\n\n'Provoked': Indica que el ataque ocurrió como respuesta a una provocación o interacción directa con el tiburón por parte \nde la persona, como tocarlo o molestarlo de alguna manera.\n\n'Questionable': Se utiliza cuando la categorización del ataque de tiburón no está clara y se requiere una mayor investigación \no información para determinar su naturaleza.\n\n'Sea Disaster': Este valor se aplica cuando el incidente no fue un ataque de tiburón directo, sino más bien una situación en \nla que muchas personas estuvieron involucradas y / o personas murieron debi

In [41]:
# Cambiar questionable por un nan
df['Type'].value_counts()

Unprovoked      3959
Provoked         504
Invalid          476
Boat             282
Sea Disaster     200
Questionable       2
Name: Type, dtype: int64

In [42]:
df['Type'] = np.where(df['Type'].isin(['Questionable']), np.nan, df['Type'])
df['Type'].unique()

array(['Boat', 'Unprovoked', 'Invalid', 'Provoked', nan, 'Sea Disaster'],
      dtype=object)

In [43]:
valores = ['Invalid', 'Sea Disaster']
df_filtrado = df.loc[df['Type'].isin(valores)] # localización de los valores 'Invalid', 'Sea Disaster', para ver si han
# sido causados por ataques de tiburones
df_filtrado

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
2,2018-06-09,2018-06-09,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-09,2018-06-09,6301.0,NaN,NaN
13,2018-05-13,2018-05-13,2018.0,Invalid,ENGLAND,Cornwall,Off Land's End,Fishing,Max Berryman,M,21,Injured by teeth of a dead porbeagle shark he ...,N,08h15,Invalid incident,"K. McMurray, TrackingSharks.com",2018.05.13.a-Berryman.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-05-13,2018-05-13,6290.0,NaN,NaN
31,2018-04-10,2018-04-10,2018.0,Invalid,BRAZIL,Alagoas,"Praia de Sauaçuhy, Maceió",Fishing,Josias Paz,M,56,Injury to ankle from marine animal trapped in ...,N,NaN,Shark involvement not confirmed,"K. McMurray, TrackingSharks.com",2018.04.10.R-Paz.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-04-10,2018-04-10,6272.0,NaN,NaN
43,2018-02-14,2018-02-14,2018.0,Invalid,AUSTRALIA,Queensland,Mooloolaba Beach,Swimming,Sharna Babd,F,NaN,"Collision / No injury, no attack",N,18h00,Possibly a wobbegong,"Sunshine Coast Daily, 2/15/2018",2018.02.14-Babd.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-02-14,2018-02-14,6260.0,NaN,NaN
44,2018-02-11,2018-02-11,2018.0,Invalid,BRAZIL,"Boi Island, Victoria",Espirito Santo,Cleaning fish,Rosalida Souza,F,46,Lacerations to 4 toes of right foot,N,NaN,"Injury believed caused by an eel, not a shark",TrackingSharks.com,2018.02.11-Rosilda.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-02-11,2018-02-11,6259.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6111,1818-05-22,1818-05-22,1818.0,Invalid,NORWAY,NaN,Herøy,NaN,male,M,NaN,Probable drowning,NaN,NaN,Shark involvement prior to death unconfirmed,"C. Moore, GSAF",1818.05.22.R-Norway.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1818-05-22,1818-05-22,192.0,NaN,NaN
6122,1808-06-01,1808-06-01,1808.0,Sea Disaster,ANTIGUA,NaN,Falmouth Harbour,NaN,NaN,NaN,NaN,Probable drowning & scavenging,NaN,NaN,NaN,"London Observer, 5/1/1808",1808.05.01.R-Antigua.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1808-05-01,1808-06-01,181.0,NaN,NaN
6125,1806-04-24,1806-04-24,1896.0,Sea Disaster,USA,NaN,NaN,NaN,Crew of the schooner Mary,M,NaN,Shark scavenged on the dead sailors,NaN,NaN,NaN,"Evening Post, 9/6/1806",1806.04.24-crew-of-the-Mary.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1806-04-24,1806-04-24,178.0,NaN,NaN
6128,1803-05-17,1803-05-17,1803.0,Sea Disaster,USA,South Carolina,Off Charleston,NaN,Captain Jones,M,NaN,No injury,N,NaN,NaN,"Evening Post, 6/13/1803",1803.05.17-Jones.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1803-05-17,1803-05-17,175.0,NaN,NaN


# 3. Limpiar Country, Location y Area

## 3.1 Country

In [44]:
df['Country'].unique()

array(['USA', 'AUSTRALIA', 'MEXICO', 'BRAZIL', 'ENGLAND', 'SOUTH AFRICA',
       'THAILAND', 'COSTA RICA', 'MALDIVES', 'BAHAMAS', 'NEW CALEDONIA',
       'ECUADOR', 'MALAYSIA', 'LIBYA', nan, 'CUBA', 'MAURITIUS',
       'NEW ZEALAND', 'SPAIN', 'SAMOA', 'SOLOMON ISLANDS', 'JAPAN',
       'EGYPT', 'ST HELENA, British overseas territory', 'COMOROS',
       'REUNION', 'FRENCH POLYNESIA', 'UNITED KINGDOM',
       'UNITED ARAB EMIRATES', 'PHILIPPINES', 'INDONESIA', 'CHINA',
       'COLUMBIA', 'CAPE VERDE', 'Fiji', 'DOMINICAN REPUBLIC',
       'CAYMAN ISLANDS', 'ARUBA', 'MOZAMBIQUE', 'PUERTO RICO', 'ITALY',
       'ATLANTIC OCEAN', 'GREECE', 'ST. MARTIN', 'FRANCE',
       'PAPUA NEW GUINEA', 'TRINIDAD & TOBAGO', 'KIRIBATI', 'ISRAEL',
       'DIEGO GARCIA', 'TAIWAN', 'JAMAICA', 'PALESTINIAN TERRITORIES',
       'GUAM', 'SEYCHELLES', 'BELIZE', 'NIGERIA', 'TONGA', 'SCOTLAND',
       'CANADA', 'CROATIA', 'FIJI', 'SAUDI ARABIA', 'CHILE', 'ANTIGUA',
       'KENYA', 'RUSSIA', 'TURKS & CAICOS', 'UNITE

In [45]:
def clean_strings(x):
    x = str(x) # convertir a string
    pattern= re.findall(r'[a-zA-Z\(\)\-\?]+', x) # buscar patron regex
    if x == 'Unknown':
        return None
    elif pattern:
        return x.lower()

In [46]:
df['Country'] = df['Country'].apply(clean_strings)
df['Country']

0             usa
1             usa
2             usa
3       australia
4          mexico
          ...    
6150      jamaica
6151          usa
6155          nan
6156        italy
6160     barbados
Name: Country, Length: 5427, dtype: object

In [47]:
df['Country'] = df['Country'].str.replace(r'[(\)\-\?]+', '')

In [48]:
df['Country'].unique()

array(['usa', 'australia', 'mexico', 'brazil', 'england', 'south africa',
       'thailand', 'costa rica', 'maldives', 'bahamas', 'new caledonia',
       'ecuador', 'malaysia', 'libya', 'nan', 'cuba', 'mauritius',
       'new zealand', 'spain', 'samoa', 'solomon islands', 'japan',
       'egypt', 'st helena, british overseas territory', 'comoros',
       'reunion', 'french polynesia', 'united kingdom',
       'united arab emirates', 'philippines', 'indonesia', 'china',
       'columbia', 'cape verde', 'fiji', 'dominican republic',
       'cayman islands', 'aruba', 'mozambique', 'puerto rico', 'italy',
       'atlantic ocean', 'greece', 'st. martin', 'france',
       'papua new guinea', 'trinidad & tobago', 'kiribati', 'israel',
       'diego garcia', 'taiwan', 'jamaica', 'palestinian territories',
       'guam', 'seychelles', 'belize', 'nigeria', 'tonga', 'scotland',
       'canada', 'croatia', 'saudi arabia', 'chile', 'antigua', 'kenya',
       'russia', 'turks & caicos', 'united arab

## 3.2 Area

In [49]:
df['Area'].unique()

array(['California', 'Georgia', 'Hawaii', 'New South Wales', 'Colima',
       'Pernambuco', 'Florida', 'Queensland', 'South Carolina',
       'Cornwall', 'Eastern Cape Province', 'Hua Hin', 'Cocos Island',
       'Western Australia', 'Alifu Alifu Atoll', 'Western Cape Province',
       'New Providence', 'Alagoas', nan, 'Victoria', 'KwaZulu-Natal',
       'Boi Island, Victoria', 'Galapagos Islands', 'Fernando de Noronha',
       'Sepang', 'Holquin Province', 'Pamplemousses ', 'South Australia',
       'North Island', 'New York', 'Canary Islands', ' Upolu Island',
       'Westerm Australia', 'Shizuoka Prefecture', 'Texas', 'Castellón',
       'Massachusetts', 'Red Sea Protectorate', 'New Providence District',
       '40 miles off Grand Bahama Island', 'Ascension Island',
       'New Jersey', 'Majorca', 'Washington', 'Tabasco', 'Anjouan',
       'Ibiza Island', 'Marquesas', 'South Devon', 'New Providence ',
       'Sharjah, ', 'Baja California Sur', 'Saint-Leu', 'South Island',
       'Lu

In [50]:
def clean_location(df, columna):
    df[columna] = df[columna].astype(str)
    df[columna] = df[columna].apply(lambda x: re.sub(r'(\d+|º)', '', x))  # eliminar datos numéricos y símbolo º
    df[columna] = df[columna].apply(lambda x: re.sub(r'\b(west|north|south|east)\b', '', x, flags=re.IGNORECASE))  # eliminar palabras "west", "north", "south" y "east"
    df[columna] = df[columna].apply(lambda x: x.strip())  # eliminar espacios en blanco al inicio o final
    df[columna] = np.where(df[columna].str.isnumeric(), np.nan, df[columna])  # valores numéricos como NaN
    return df

In [51]:
clean_location(df, 'Area')

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018-06-25,2018-06-25,2018.0,Boat,usa,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-25,2018-06-25,6303.0,NaN,NaN
1,2018-06-18,2018-06-18,2018.0,Unprovoked,usa,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-18,2018-06-18,6302.0,NaN,NaN
2,2018-06-09,2018-06-09,2018.0,Invalid,usa,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-09,2018-06-09,6301.0,NaN,NaN
3,2018-06-08,2018-06-08,2018.0,Unprovoked,australia,New Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-08,2018-06-08,6300.0,NaN,NaN
4,2018-06-04,2018-06-04,2018.0,Provoked,mexico,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-04,2018-06-04,6299.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6150,1753-10-27,1753-10-27,1753.0,Unprovoked,jamaica,nan,Kingston Harbor,Fell overboard,male,M,NaN,FATAL,Y,NaN,NaN,"Maryland Gazette, 1/31/1754",1753.10.27.R-Jamaica.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1753-10-27,1753-10-27,153.0,NaN,NaN
6151,1751-07-27,1751-07-27,1751.0,Unprovoked,usa,Massachusetts,NaN,Swimming,male,M,NaN,FATAL,Y,NaN,NaN,"Pennsylvania Gazette, 8/15/1751",1751.07.27-Massachusetts.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1751-07-27,1751-07-27,152.0,NaN,NaN
6155,1742-12-17,1742-12-17,1742.0,Unprovoked,nan,nan,Carlisle Bay,Swimming,2 impressed seamen,M,NaN,FATAL,Y,NaN,NaN,"C. Moore, GSAF",1742.12.17-AdviceSeamen.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1742-12-17,1742-12-17,148.0,NaN,NaN
6156,1738-04-06,1738-04-06,1738.0,Unprovoked,italy,Sicily,Strait of Messina,Swimming,male,M,NaN,FATAL,Y,NaN,NaN,"C. Moore, GSAF",1738.04.06.R-Messina.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1738-04-06,1738-04-06,147.0,NaN,NaN


In [52]:
valores_a_buscar = ['N, E', 'N, W', '.N .W', 'S, E']
df.loc[df['Area'].isin(valores_a_buscar), 'Area'] = np.nan

df['Area'].unique()

array(['California', 'Georgia', 'Hawaii', 'New  Wales', 'Colima',
       'Pernambuco', 'Florida', 'Queensland', 'Carolina', 'Cornwall',
       'Eastern Cape Province', 'Hua Hin', 'Cocos Island',
       'Western Australia', 'Alifu Alifu Atoll', 'Western Cape Province',
       'New Providence', 'Alagoas', 'nan', 'Victoria', 'KwaZulu-Natal',
       'Boi Island, Victoria', 'Galapagos Islands', 'Fernando de Noronha',
       'Sepang', 'Holquin Province', 'Pamplemousses', 'Australia',
       'Island', 'New York', 'Canary Islands', 'Upolu Island',
       'Westerm Australia', 'Shizuoka Prefecture', 'Texas', 'Castellón',
       'Massachusetts', 'Red Sea Protectorate', 'New Providence District',
       'miles off Grand Bahama Island', 'Ascension Island', 'New Jersey',
       'Majorca', 'Washington', 'Tabasco', 'Anjouan', 'Ibiza Island',
       'Marquesas', 'Devon', 'Sharjah,', 'Baja California Sur',
       'Saint-Leu', 'Luzon Island', 'Great Exuma', 'Saint-Andre',
       'Bimini', 'Bali', 'Tuamot

## 3.3 Location

In [53]:
df['Location'].unique()

array(['Oceanside, San Diego County', 'St. Simon Island, Glynn County',
       'Habush, Oahu', ..., 'Damiscotte', 'Carlisle Bay',
       'Strait of Messina'], dtype=object)

In [54]:
len(df['Location'].unique())

3632

In [55]:
df['Location'] = df['Location'].str.replace(r'[(\)\-\?]+', '')

# 4. Limpieza Activity e Injury

## 4.1 Activity

In [56]:
df['Activity'].unique()

array(['Paddling', 'Standing', 'Surfing', ...,
       "Standing on landed shark's tail", 'Dropped overboard',
       'Washing himself'], dtype=object)

In [57]:
len(df['Activity'].unique())

1277

In [58]:
df['Activity'] = df['Activity'].apply(clean_strings)

In [59]:
df['Activity'].value_counts()

surfing                                                     939
swimming                                                    789
nan                                                         387
fishing                                                     375
spearfishing                                                271
                                                           ... 
swimming near his boat                                        1
diving & force-feeding the shark                              1
attempting to drag hooked shark ashore by its tail            1
swimming underwater from crayfish cage to a fishing bait      1
washing himself                                               1
Name: Activity, Length: 1247, dtype: int64

In [60]:
def clean_activity(df, columna):
    palabras_claves = {'surfing': 'surfing', 'swimming': 'swimming', 'fishing': 'fishing', 'spearfishing': 'spearfishing', 'bathing': 'bathing'}
    
    def clasificar_valor(valor):
        for palabra, clasificacion in palabras_claves.items():
            if palabra in str(valor).lower():
                return clasificacion
        return valor
    
    df[columna] = df[columna].apply(clasificar_valor)
    return df

In [61]:
clean_activity(df, 'Activity')

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018-06-25,2018-06-25,2018.0,Boat,usa,California,"Oceanside, San Diego County",paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-25,2018-06-25,6303.0,NaN,NaN
1,2018-06-18,2018-06-18,2018.0,Unprovoked,usa,Georgia,"St. Simon Island, Glynn County",standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-18,2018-06-18,6302.0,NaN,NaN
2,2018-06-09,2018-06-09,2018.0,Invalid,usa,Hawaii,"Habush, Oahu",surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-09,2018-06-09,6301.0,NaN,NaN
3,2018-06-08,2018-06-08,2018.0,Unprovoked,australia,New Wales,Arrawarra Headland,surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-08,2018-06-08,6300.0,NaN,NaN
4,2018-06-04,2018-06-04,2018.0,Provoked,mexico,Colima,La Ticla,free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-04,2018-06-04,6299.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6150,1753-10-27,1753-10-27,1753.0,Unprovoked,jamaica,nan,Kingston Harbor,fell overboard,male,M,NaN,FATAL,Y,NaN,NaN,"Maryland Gazette, 1/31/1754",1753.10.27.R-Jamaica.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1753-10-27,1753-10-27,153.0,NaN,NaN
6151,1751-07-27,1751-07-27,1751.0,Unprovoked,usa,Massachusetts,NaN,swimming,male,M,NaN,FATAL,Y,NaN,NaN,"Pennsylvania Gazette, 8/15/1751",1751.07.27-Massachusetts.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1751-07-27,1751-07-27,152.0,NaN,NaN
6155,1742-12-17,1742-12-17,1742.0,Unprovoked,nan,nan,Carlisle Bay,swimming,2 impressed seamen,M,NaN,FATAL,Y,NaN,NaN,"C. Moore, GSAF",1742.12.17-AdviceSeamen.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1742-12-17,1742-12-17,148.0,NaN,NaN
6156,1738-04-06,1738-04-06,1738.0,Unprovoked,italy,Sicily,Strait of Messina,swimming,male,M,NaN,FATAL,Y,NaN,NaN,"C. Moore, GSAF",1738.04.06.R-Messina.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1738-04-06,1738-04-06,147.0,NaN,NaN


In [62]:
df['Activity'].value_counts().head(20)

surfing            1087
swimming            992
fishing             972
nan                 387
bathing             161
wading              137
diving               92
standing             89
snorkeling           83
scuba diving         83
body boarding        64
boogie boarding      43
kayaking             33
treading water       29
free diving          25
fell overboard       24
pearl diving         23
walking              16
sea disaster         13
floating             12
Name: Activity, dtype: int64

In [63]:
def categorize_string_1(string):
    keywords = ['walking', 'standing', 'wading']
    category = 'wading'
    
    for keyword in keywords:
        if keyword in str(string).lower():
            return category
    
    return string

In [64]:
df['Activity'] = df['Activity'].apply(categorize_string_1)

In [65]:
def categorize_string_2(string):
    keywords = ['treading water', 'floating']
    category = 'treading water'
    
    for keyword in keywords:
        if keyword in str(string).lower():
            return category
    
    return string

In [66]:
df['Activity'] = df['Activity'].apply(categorize_string_2)

In [67]:
def categorize_string_3(string):
    keywords = ['body boarding', 'boogie boarding']
    category = 'body boarding'
    
    for keyword in keywords:
        if keyword in str(string).lower():
            return category
    
    return string

In [68]:
df['Activity'] = df['Activity'].apply(categorize_string_3)

In [69]:
def categorize_string_4(string):
    keywords = ['pearl diving']
    category = 'diving'
    
    for keyword in keywords:
        if keyword in str(string).lower():
            return category
    
    return string

In [70]:
df['Activity'] = df['Activity'].apply(categorize_string_4)

In [71]:
def categorize_string_5(string):
    keywords = ['surf skiing', 'surf-skiing']
    category = 'surf skiing'
    
    for keyword in keywords:
        if keyword in str(string).lower():
            return category
    
    return string

In [72]:
df['Activity'] = df['Activity'].apply(categorize_string_5)

In [73]:
def categorize_string_6(string):
    keywords = ['paddle boarding', 'paddle-skiing']
    category = 'paddle boarding'
    
    for keyword in keywords:
        if keyword in str(string).lower():
            return category
    
    return string

In [74]:
df['Activity'] = df['Activity'].apply(categorize_string_6)

In [75]:
df['Activity'].value_counts().head(20)

surfing                 1087
swimming                 992
fishing                  972
nan                      387
wading                   287
bathing                  161
diving                   117
body boarding            109
snorkeling                83
scuba diving              83
treading water            76
kayaking                  33
surf skiing               28
free diving               25
fell overboard            24
sea disaster              13
rowing                    11
canoeing                  11
paddle boarding           10
sitting on surfboard       9
Name: Activity, dtype: int64

In [76]:
len(df['Activity'].unique())

734

## 4.2 Injury

In [77]:
df['Injury'].unique()

array(['No injury to occupant, outrigger canoe and paddle damaged',
       'Minor injury to left thigh',
       'Injury to left lower leg from surfboard skeg', ...,
       'Human remains recovered from shark',
       'Many of the crew were taken by sharks',
       'Hand and foot severely bitten, surgically amputated'],
      dtype=object)

In [78]:
len(df['Injury'].unique())

3311

In [79]:
df['Activity'] = df['Activity'].apply(clean_strings)

# 5. Limpieza Name

In [80]:
df['Name'].unique()

array(['Julie Wolfe', 'Adyson\xa0McNeely ', 'John Denges', ...,
       'Joseph Feron', '2 impressed seamen',
       'Samuel Jennings, a deserter from the British frigate Milford'],
      dtype=object)

In [81]:
len(df['Name'].unique())

4655

In [82]:
df['Name'] = np.where(df['Name'].str.lower().isin(['male', 'female']), np.nan, df['Name'])
df.head(20)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018-06-25,2018-06-25,2018.0,Boat,usa,California,"Oceanside, San Diego County",paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-25,2018-06-25,6303.0,NaN,NaN
1,2018-06-18,2018-06-18,2018.0,Unprovoked,usa,Georgia,"St. Simon Island, Glynn County",wading,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-18,2018-06-18,6302.0,NaN,NaN
2,2018-06-09,2018-06-09,2018.0,Invalid,usa,Hawaii,"Habush, Oahu",surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-09,2018-06-09,6301.0,NaN,NaN
3,2018-06-08,2018-06-08,2018.0,Unprovoked,australia,New Wales,Arrawarra Headland,surfing,NaN,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-08,2018-06-08,6300.0,NaN,NaN
4,2018-06-04,2018-06-04,2018.0,Provoked,mexico,Colima,La Ticla,free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-04,2018-06-04,6299.0,NaN,NaN
5,2018-06-03,2018-06-03,2018.0,Unprovoked,australia,New Wales,"Flat Rock, Ballina",surfing,Chris,M,NaN,"No injury, board bitten",N,NaN,NaN,"Daily Telegraph, 6/4/2018",2018.06.03.b-FlatRock.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-03,2018-06-03,6298.0,NaN,NaN
6,2018-06-03,2018-06-03,2018.0,Unprovoked,brazil,Pernambuco,"Piedade Beach, Recife",swimming,Jose Ernesto da Silva,M,18,FATAL,Y,Late afternoon,Tiger shark,"Diario de Pernambuco, 6/4/2018",2018.06.03.a-daSilva.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-03,2018-06-03,6297.0,NaN,NaN
7,2018-05-27,2018-05-27,2018.0,Unprovoked,usa,Florida,"Lighhouse Point Park, Ponce Inlet, Volusia County",fishing,NaN,M,52,Minor injury to foot. PROVOKED INCIDENT,N,NaN,"Lemon shark, 3'","K. McMurray, TrackingSharks.com",2018.05.27-Ponce.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-05-27,2018-05-27,6296.0,NaN,NaN
8,2018-05-26,2018-05-26,2018.0,Unprovoked,usa,Florida,"Cocoa Beach, Brevard County",wading,Cody High,M,15,Lower left leg bitten,N,17h00,"Bull shark, 6'","K.McMurray, TrackingSharks.com",2018.05.26.b-High.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-05-26,2018-05-26,6295.0,NaN,NaN
9,2018-05-26,2018-05-26,2018.0,Unprovoked,usa,Florida,"Daytona Beach, Volusia County",wading,NaN,M,12,Minor injury to foot,N,14h00,NaN,"K. McMurray, Tracking Sharks.com",2018.05.26.a-DaytonaBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-05-26,2018-05-26,6294.0,NaN,NaN


# 8. Limpieza Sex y Age

## 8.1 Sex

In [83]:
df['Sex '].unique()

array(['F', 'M', nan, 'M ', 'lli', 'N', '.'], dtype=object)

In [84]:
def limpiar_genero(df, columna):
    df[columna] = df[columna].astype(str)
    df[columna] = df[columna].apply(lambda x: re.sub(r'[^\w\s]','',x))  # eliminar todos los símbolos
    df[columna] = df[columna].apply(lambda x: re.sub(r'\s+','',x))  # eliminar espacios en blanco
    df[columna] = np.where(df[columna].str.contains('^F|^f|^Fe|^fe', regex=True), 'F',
                           np.where(df[columna].str.contains('^M|^m|^Ma|^ma', regex=True), 'M', np.nan))
    return df
    

In [85]:
limpiar_genero(df, 'Sex ')

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018-06-25,2018-06-25,2018.0,Boat,usa,California,"Oceanside, San Diego County",paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-25,2018-06-25,6303.0,NaN,NaN
1,2018-06-18,2018-06-18,2018.0,Unprovoked,usa,Georgia,"St. Simon Island, Glynn County",wading,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-18,2018-06-18,6302.0,NaN,NaN
2,2018-06-09,2018-06-09,2018.0,Invalid,usa,Hawaii,"Habush, Oahu",surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-09,2018-06-09,6301.0,NaN,NaN
3,2018-06-08,2018-06-08,2018.0,Unprovoked,australia,New Wales,Arrawarra Headland,surfing,NaN,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-08,2018-06-08,6300.0,NaN,NaN
4,2018-06-04,2018-06-04,2018.0,Provoked,mexico,Colima,La Ticla,free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-04,2018-06-04,6299.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6150,1753-10-27,1753-10-27,1753.0,Unprovoked,jamaica,nan,Kingston Harbor,fell overboard,NaN,M,NaN,FATAL,Y,NaN,NaN,"Maryland Gazette, 1/31/1754",1753.10.27.R-Jamaica.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1753-10-27,1753-10-27,153.0,NaN,NaN
6151,1751-07-27,1751-07-27,1751.0,Unprovoked,usa,Massachusetts,NaN,swimming,NaN,M,NaN,FATAL,Y,NaN,NaN,"Pennsylvania Gazette, 8/15/1751",1751.07.27-Massachusetts.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1751-07-27,1751-07-27,152.0,NaN,NaN
6155,1742-12-17,1742-12-17,1742.0,Unprovoked,nan,nan,Carlisle Bay,swimming,2 impressed seamen,M,NaN,FATAL,Y,NaN,NaN,"C. Moore, GSAF",1742.12.17-AdviceSeamen.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1742-12-17,1742-12-17,148.0,NaN,NaN
6156,1738-04-06,1738-04-06,1738.0,Unprovoked,italy,Sicily,Strait of Messina,swimming,NaN,M,NaN,FATAL,Y,NaN,NaN,"C. Moore, GSAF",1738.04.06.R-Messina.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1738-04-06,1738-04-06,147.0,NaN,NaN


In [86]:
df['Sex '].unique()

array(['F', 'M', 'nan'], dtype=object)

## 8.2 Age

In [87]:
df['Age'].unique()

array(['57', '11', '48', nan, '18', '52', '15', '12', '32', '10', '21',
       '30', '60', '33', '29', '54', '34', '41', '37', '56', '19', '25',
       '69', '38', '55', '35', '46', '45', '14', '40s', '28', '20', '24',
       '26', '49', '22', '7', '31', '17', '40', '13', '42', '3', '8',
       '50', '16', '82', '73', '20s', '68', '51', '39', '58', 'Teen',
       '47', '61', '65', '36', '66', '43', '60s', '9', '72', '59', '6',
       '27', '64', '23', '71', '44', '62', '63', '70', '18 months', '53',
       '30s', '50s', 'teen', '77', '74', '28 & 26', '5', '86', '18 or 20',
       '12 or 13', '46 & 34', '28, 23 & 30', 'Teens', '36 & 26',
       '8 or 10', '84', '\xa0 ', ' ', '30 or 36', '6½', '21 & ?', '75',
       '33 or 37', 'mid-30s', '23 & 20', ' 30', '7      &    31', ' 28',
       '20?', '32 & 30', '16 to 18', '87', '67', 'Elderly', 'mid-20s',
       '74 ', '45 ', '21 or 26', '20 ', '>50', '18 to 22', 'adult',
       '9 & 12', '? & 19', '(adult)', '33 & 37', '25 or 28',
       '37

In [88]:
def age_filt(x):
    x=str(x)
    age_pattern = r"\b(\d{2})\b"  # Extracts two-digit numbers
    match = re.search(age_pattern, x)
    
    if match:
        age = int(match.group(1))
        return int(age)
    else:
        return np.nan

In [89]:
df['Age'] = df['Age'].apply(age_filt)
df['Age']

0       57.0
1       11.0
2       48.0
3        NaN
4        NaN
        ... 
6150     NaN
6151     NaN
6155     NaN
6156     NaN
6160    19.0
Name: Age, Length: 5427, dtype: float64

In [90]:
df['Age'].unique()

array([57., 11., 48., nan, 18., 52., 15., 12., 32., 10., 21., 30., 60.,
       33., 29., 54., 34., 41., 37., 56., 19., 25., 69., 38., 55., 35.,
       46., 45., 14., 28., 20., 24., 26., 49., 22., 31., 17., 40., 13.,
       42., 50., 16., 82., 73., 68., 51., 39., 58., 47., 61., 65., 36.,
       66., 43., 72., 59., 27., 64., 23., 71., 44., 62., 63., 70., 53.,
       77., 74., 86., 84., 75., 87., 67., 81., 78.])

# 9. Limpieza 'Fatal'

In [91]:
df['Fatal (Y/N)'].unique()

array(['N', 'Y', nan, 'M', 'UNKNOWN', '2017', ' N'], dtype=object)

In [92]:
df['Fatal (Y/N)'].value_counts()

N          3806
Y          1103
UNKNOWN      42
 N            5
M             1
2017          1
Name: Fatal (Y/N), dtype: int64

In [93]:
def clean_fatal(x):
    x=str(x)
    pattern1 = r"[nN]"
    pattern2 = r"[yY]"

    if x == 'UNKNOWN':
        
        return np.nan
    
    elif re.findall(pattern1,x):
        
        return False
    
    elif re.findall(pattern2,x):
        
        return True
        

In [94]:
df['Fatal (Y/N)'] = df['Fatal (Y/N)'].apply(clean_fatal)
df['Fatal (Y/N)'].unique()

array([False, True, None, nan], dtype=object)

In [95]:
df['Fatal (Y/N)'].value_counts()

False    4280
True     1103
Name: Fatal (Y/N), dtype: int64

# 10. Limpieza Time

In [96]:
df['Time'].unique()

array(['18h00', '14h00  -15h00', '07h45', nan, 'Late afternoon', '17h00',
       '14h00', 'Morning', '15h00', '08h15', '11h00', '10h30', '10h40',
       '16h50', '07h00', '09h30', 'Afternoon', '21h50', '09h40', '08h00',
       '17h35', '15h30', '07h30', '19h00, Dusk', 'Night', '16h00',
       '15h01', '12h00', '13h45', '23h30', '09h00', '14h30', '18h30',
       '12h30', '16h30', '18h45', '06h00', '10h00', '10h44', '13h19',
       'Midday', '13h30', '10h45', '11h20', '11h45', '19h30', '08h30',
       '15h45', 'Shortly before 12h00', '17h34', '17h10', '11h15',
       '08h50', '17h45', '13h00', '10h20', '13h20', '02h00', '09h50',
       '11h30', '17h30', '9h00', '10h43', 'After noon', '15h15', '15h40',
       '19h05', '1300', '14h30 / 15h30', '22h00', '16h20', '14h34',
       '15h25', '14h55', '17h46', 'Morning ', '15h49', '19h00',
       'Midnight', '09h30 / 10h00', '10h15', '18h15', '04h00', '14h50',
       '13h50', '19h20', '10h25', '10h45-11h15', '16h45', '15h52',
       '06h15', '14h

In [97]:
df.head(20)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018-06-25,2018-06-25,2018.0,Boat,usa,California,"Oceanside, San Diego County",paddling,Julie Wolfe,F,57.0,"No injury to occupant, outrigger canoe and pad...",False,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-25,2018-06-25,6303.0,NaN,NaN
1,2018-06-18,2018-06-18,2018.0,Unprovoked,usa,Georgia,"St. Simon Island, Glynn County",wading,Adyson McNeely,F,11.0,Minor injury to left thigh,False,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-18,2018-06-18,6302.0,NaN,NaN
2,2018-06-09,2018-06-09,2018.0,Invalid,usa,Hawaii,"Habush, Oahu",surfing,John Denges,M,48.0,Injury to left lower leg from surfboard skeg,False,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-09,2018-06-09,6301.0,NaN,NaN
3,2018-06-08,2018-06-08,2018.0,Unprovoked,australia,New Wales,Arrawarra Headland,surfing,NaN,M,NaN,Minor injury to lower leg,False,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-08,2018-06-08,6300.0,NaN,NaN
4,2018-06-04,2018-06-04,2018.0,Provoked,mexico,Colima,La Ticla,free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,False,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-04,2018-06-04,6299.0,NaN,NaN
5,2018-06-03,2018-06-03,2018.0,Unprovoked,australia,New Wales,"Flat Rock, Ballina",surfing,Chris,M,NaN,"No injury, board bitten",False,NaN,NaN,"Daily Telegraph, 6/4/2018",2018.06.03.b-FlatRock.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-03,2018-06-03,6298.0,NaN,NaN
6,2018-06-03,2018-06-03,2018.0,Unprovoked,brazil,Pernambuco,"Piedade Beach, Recife",swimming,Jose Ernesto da Silva,M,18.0,FATAL,True,Late afternoon,Tiger shark,"Diario de Pernambuco, 6/4/2018",2018.06.03.a-daSilva.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-03,2018-06-03,6297.0,NaN,NaN
7,2018-05-27,2018-05-27,2018.0,Unprovoked,usa,Florida,"Lighhouse Point Park, Ponce Inlet, Volusia County",fishing,NaN,M,52.0,Minor injury to foot. PROVOKED INCIDENT,False,NaN,"Lemon shark, 3'","K. McMurray, TrackingSharks.com",2018.05.27-Ponce.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-05-27,2018-05-27,6296.0,NaN,NaN
8,2018-05-26,2018-05-26,2018.0,Unprovoked,usa,Florida,"Cocoa Beach, Brevard County",wading,Cody High,M,15.0,Lower left leg bitten,False,17h00,"Bull shark, 6'","K.McMurray, TrackingSharks.com",2018.05.26.b-High.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-05-26,2018-05-26,6295.0,NaN,NaN
9,2018-05-26,2018-05-26,2018.0,Unprovoked,usa,Florida,"Daytona Beach, Volusia County",wading,NaN,M,12.0,Minor injury to foot,False,14h00,NaN,"K. McMurray, Tracking Sharks.com",2018.05.26.a-DaytonaBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-05-26,2018-05-26,6294.0,NaN,NaN


In [98]:
def limpiar_hora(hora):
    if isinstance(hora, str):
        hora = hora.strip().lower()
        if 'between' in hora:
            hora = hora.split('between')[-1].strip()
        if 'sometime' in hora:
            hora = hora.split('sometime')[-1].strip()

        # Limpieza del formato hhoo
        if 'oo' in hora:
            hora = hora.replace('oo', '00')
        
        # Limpieza del formato hhmm
        if hora.endswith('h') or hora.endswith('hh'):
            hora = hora[:-1].strip()
            if len(hora) < 4:
                hora = hora.zfill(4)
        
        if hora.startswith('just before') or hora.startswith('just after'):
            hora = hora.split('before')[-1].strip()
            hora = hora.split('after')[-1].strip()
        
        # Limpieza de otros formatos
        formatos_fecha = ['%Hh%M', '%Hh%M:%S', '%H:%M']
        for formato in formatos_fecha:
            try:
                hora_dt = pd.to_datetime(hora, format=formato)
                return hora_dt.time()
            except ValueError:
                continue
    
    return np.nan

In [99]:
df['Time'] = df['Time'].apply(limpiar_hora)
df['Time'].head(20)

0     18:00:00
1          NaN
2     07:45:00
3          NaN
4          NaN
5          NaN
6          NaN
7          NaN
8     17:00:00
9     14:00:00
10         NaN
11         NaN
12    15:00:00
13    08:15:00
15    11:00:00
16    10:30:00
17         NaN
18    10:40:00
19    16:50:00
20    07:00:00
Name: Time, dtype: object

# 11. Limpieza Species

In [100]:
df['Species '].unique()

array(['White shark', nan, '2 m shark', ..., '234-lb shark',
       "Said to involve a 6 m to 7.3 m [20' to 24'] shark",
       "Said to be a 7.6 m [25'] shark"], dtype=object)

In [101]:
len(df['Species '].unique())

1424

In [102]:
df['Species '].value_counts()

White shark                                           144
Shark involvement prior to death was not confirmed     99
Invalid                                                84
Shark involvement not confirmed                        81
Shark involvement prior to death unconfirmed           58
                                                     ... 
Shark involvement prior to death unconfired             1
Spinner shark, 3' to 4'                                 1
Nurse shark, 1m                                         1
Blue shark 6'                                           1
Said to be a 7.6 m [25'] shark                          1
Name: Species , Length: 1423, dtype: int64

In [103]:
df['Species '] = np.where(df['Species '].isin(['Shark involvement prior to death was not confirmed', 'Shark involvement not confirmed', 'Shark involvement prior to death unconfirmed', 'Shark involvement prior to death unconfired', 'Shark involvement not confirmed, injury may be due to a stingray', 'Questionable']), np.nan, df['Species '])
df['Species '].value_counts()

White shark                                     144
Invalid                                          84
Tiger shark                                      56
Bull shark                                       41
4' shark                                         40
                                               ... 
3 m to 4.5 m [10' to 15'] white shark             1
Wobbegong shark, 1m                               1
White shark, 4 m white shark                      1
Thought to involve a >2 m [6.75'] bull shark      1
Said to be a 7.6 m [25'] shark                    1
Name: Species , Length: 1417, dtype: int64

In [104]:
len(df['Species '].unique())

1418

In [105]:
df.head(20)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018-06-25,2018-06-25,2018.0,Boat,usa,California,"Oceanside, San Diego County",paddling,Julie Wolfe,F,57.0,"No injury to occupant, outrigger canoe and pad...",False,18:00:00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-25,2018-06-25,6303.0,NaN,NaN
1,2018-06-18,2018-06-18,2018.0,Unprovoked,usa,Georgia,"St. Simon Island, Glynn County",wading,Adyson McNeely,F,11.0,Minor injury to left thigh,False,NaN,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-18,2018-06-18,6302.0,NaN,NaN
2,2018-06-09,2018-06-09,2018.0,Invalid,usa,Hawaii,"Habush, Oahu",surfing,John Denges,M,48.0,Injury to left lower leg from surfboard skeg,False,07:45:00,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-09,2018-06-09,6301.0,NaN,NaN
3,2018-06-08,2018-06-08,2018.0,Unprovoked,australia,New Wales,Arrawarra Headland,surfing,NaN,M,NaN,Minor injury to lower leg,False,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-08,2018-06-08,6300.0,NaN,NaN
4,2018-06-04,2018-06-04,2018.0,Provoked,mexico,Colima,La Ticla,free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,False,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-04,2018-06-04,6299.0,NaN,NaN
5,2018-06-03,2018-06-03,2018.0,Unprovoked,australia,New Wales,"Flat Rock, Ballina",surfing,Chris,M,NaN,"No injury, board bitten",False,NaN,NaN,"Daily Telegraph, 6/4/2018",2018.06.03.b-FlatRock.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-03,2018-06-03,6298.0,NaN,NaN
6,2018-06-03,2018-06-03,2018.0,Unprovoked,brazil,Pernambuco,"Piedade Beach, Recife",swimming,Jose Ernesto da Silva,M,18.0,FATAL,True,NaN,Tiger shark,"Diario de Pernambuco, 6/4/2018",2018.06.03.a-daSilva.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-03,2018-06-03,6297.0,NaN,NaN
7,2018-05-27,2018-05-27,2018.0,Unprovoked,usa,Florida,"Lighhouse Point Park, Ponce Inlet, Volusia County",fishing,NaN,M,52.0,Minor injury to foot. PROVOKED INCIDENT,False,NaN,"Lemon shark, 3'","K. McMurray, TrackingSharks.com",2018.05.27-Ponce.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-05-27,2018-05-27,6296.0,NaN,NaN
8,2018-05-26,2018-05-26,2018.0,Unprovoked,usa,Florida,"Cocoa Beach, Brevard County",wading,Cody High,M,15.0,Lower left leg bitten,False,17:00:00,"Bull shark, 6'","K.McMurray, TrackingSharks.com",2018.05.26.b-High.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-05-26,2018-05-26,6295.0,NaN,NaN
9,2018-05-26,2018-05-26,2018.0,Unprovoked,usa,Florida,"Daytona Beach, Volusia County",wading,NaN,M,12.0,Minor injury to foot,False,14:00:00,NaN,"K. McMurray, Tracking Sharks.com",2018.05.26.a-DaytonaBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-05-26,2018-05-26,6294.0,NaN,NaN


In [106]:
df.shape

(5427, 24)

In [107]:
nan_cols = df.isna().sum()

nan_cols[nan_cols>0]

Case Number                  5
Year                         2
Type                         6
Area                         7
Location                   358
Name                       648
Age                       2274
Injury                      16
Fatal (Y/N)                 44
Time                      3158
Species                   2572
Investigator or Source      12
href formula                 1
Case Number.1                6
Case Number.2                4
Unnamed: 22               5426
Unnamed: 23               5425
dtype: int64

In [108]:
df = df.dropna(subset=['Species ']) # Elimina todas las filas que contengan valores nulos en la columna Species del df

In [109]:
df.shape

(2855, 24)

In [110]:
df.reset_index(drop=True, inplace=True)

In [111]:
df.head(20)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018-06-25,2018-06-25,2018.0,Boat,usa,California,"Oceanside, San Diego County",paddling,Julie Wolfe,F,57.0,"No injury to occupant, outrigger canoe and pad...",False,18:00:00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-25,2018-06-25,6303.0,NaN,NaN
1,2018-06-08,2018-06-08,2018.0,Unprovoked,australia,New Wales,Arrawarra Headland,surfing,NaN,M,NaN,Minor injury to lower leg,False,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-08,2018-06-08,6300.0,NaN,NaN
2,2018-06-04,2018-06-04,2018.0,Provoked,mexico,Colima,La Ticla,free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,False,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-04,2018-06-04,6299.0,NaN,NaN
3,2018-06-03,2018-06-03,2018.0,Unprovoked,brazil,Pernambuco,"Piedade Beach, Recife",swimming,Jose Ernesto da Silva,M,18.0,FATAL,True,NaN,Tiger shark,"Diario de Pernambuco, 6/4/2018",2018.06.03.a-daSilva.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-03,2018-06-03,6297.0,NaN,NaN
4,2018-05-27,2018-05-27,2018.0,Unprovoked,usa,Florida,"Lighhouse Point Park, Ponce Inlet, Volusia County",fishing,NaN,M,52.0,Minor injury to foot. PROVOKED INCIDENT,False,NaN,"Lemon shark, 3'","K. McMurray, TrackingSharks.com",2018.05.27-Ponce.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-05-27,2018-05-27,6296.0,NaN,NaN
5,2018-05-26,2018-05-26,2018.0,Unprovoked,usa,Florida,"Cocoa Beach, Brevard County",wading,Cody High,M,15.0,Lower left leg bitten,False,17:00:00,"Bull shark, 6'","K.McMurray, TrackingSharks.com",2018.05.26.b-High.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-05-26,2018-05-26,6295.0,NaN,NaN
6,2018-05-24,2018-05-24,2018.0,Provoked,australia,Queensland,Cairns Aquarium,feeding sharks,NaN,M,32.0,Minor bite to hand by captive shark. PROVOKED ...,False,NaN,Grey reef shark,"ABC.net.au ,05/24/2018",2018.05.24-CairnsAquarium.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-05-24,2018-05-24,6293.0,NaN,NaN
7,2018-05-13,2018-05-13,2018.0,Invalid,england,Cornwall,Off Land's End,fishing,Max Berryman,M,21.0,Injured by teeth of a dead porbeagle shark he ...,False,08:15:00,Invalid incident,"K. McMurray, TrackingSharks.com",2018.05.13.a-Berryman.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-05-13,2018-05-13,6290.0,NaN,NaN
8,2018-04-28,2018-04-28,2018.0,Unprovoked,costa rica,Cocos Island,Manuelita,scuba diving,NaN,M,30.0,"No injury, shark bit scuba gear",False,10:40:00,Tiger shark,Costa Rica Star. 4/28/2018,2018.04.28.b-CostaRica.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-04-28,2018-04-28,6285.0,NaN,NaN
9,2018-04-25,2018-04-25,2018.0,Unprovoked,australia,Western Australia,"Surfers Point, Prevelly",surfing,Rob Bruce,M,60.0,"No injury, knocked off board by shark",False,09:30:00,3 m shark,"B.Myatt, GSAF",2018.04.25.a-Bruce.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-04-25,2018-04-25,6282.0,NaN,NaN


In [112]:
df['Species '].value_counts().head(20)

White shark              144
Invalid                   84
Tiger shark               56
Bull shark                41
4' shark                  40
6' shark                  38
1.8 m [6'] shark          32
3' shark                  25
1.5 m [5'] shark          25
5' shark                  25
4' to 5' shark            24
1.2 m [4'] shark          23
2 m shark                 22
No shark involvement      21
Questionable incident     21
3 m [10'] shark           21
3' to 4' shark            18
Wobbegong shark           17
2.4 m [8'] shark          16
Blacktip shark            15
Name: Species , dtype: int64

In [113]:
df['Species '].unique()

array(['White shark', '2 m shark', 'Tiger shark, 3m', ..., '234-lb shark',
       "Said to involve a 6 m to 7.3 m [20' to 24'] shark",
       "Said to be a 7.6 m [25'] shark"], dtype=object)

In [114]:
# Creo un dataframe nuevo para analizar las filas que no contienen la palabra 'Shark/shark' dentro de la columna 'Species'

In [115]:
rows_no_shark = df[~df['Species '].str.contains('shark|Shark', case=False)] # analiza las filas en las que no aparece la palabra Tiburón
rows_no_shark

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
7,2018-05-13,2018-05-13,2018.0,Invalid,england,Cornwall,Off Land's End,fishing,Max Berryman,M,21.0,Injured by teeth of a dead porbeagle shark he ...,False,08:15:00,Invalid incident,"K. McMurray, TrackingSharks.com",2018.05.13.a-Berryman.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-05-13,2018-05-13,6290.0,NaN,NaN
26,2018-02-14,2018-02-14,2018.0,Invalid,australia,Queensland,Mooloolaba Beach,swimming,Sharna Babd,F,NaN,"Collision / No injury, no attack",False,18:00:00,Possibly a wobbegong,"Sunshine Coast Daily, 2/15/2018",2018.02.14-Babd.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-02-14,2018-02-14,6260.0,NaN,NaN
52,2017-10-01,2017-10-01,2017.0,Invalid,south africa,Western Cape Province,Dyer Island,scuba diving,Bradley Fick,M,31.0,FATAL,False,NaN,Death may have been due to drowning,"All Africa, 10/11/2017",2017.10.01-Fick.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017-10-01,2017-10-01,6227.0,NaN,NaN
56,2017-09-16,2017-09-16,2017.0,Unprovoked,spain,Canary Islands,Gran Canaria,surfing,NaN,M,13.0,Lacerations to right foot,False,NaN,"Porbeagle, 1.5 m",Turismo La Aldea,2017.09.16.b-GrandCanary.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017-09-16,2017-09-16,6221.0,NaN,NaN
167,2016-03-03,2016-03-03,2016.0,Unprovoked,australia,Australia,Wrights Bay,fishing,Lee Taplin,M,NaN,Puncture wounds to right calf,False,NaN,Bronze whaler,"9 News, 3/1/2016",2016.03.03.R-Taplin.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016-03-03,2016-03-03,6002.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2835,1868-05-13,1868-05-13,1868.0,Unprovoked,india,Hoogly River,Ghat,wading,NaN,M,35.0,"FATAL, upper left thigh, groin & buttocks seve...",True,NaN,Identified as C. gangeticus by Dr. J. Fayrer,"J. Fayrer, M.D.",1868.05.13-Hindoo.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1868-05-13,1868-05-13,381.0,NaN,NaN
2837,1862-12-04,1862-12-04,1862.0,Invalid,new zealand,Island,Lyttleton,nan,NaN,nan,NaN,NaN,False,NaN,Questionable incident,"Bendigo Advertiser, 1/3/1863",1862.12.04-Lyttleton.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1862-12-04,1862-12-04,338.0,NaN,NaN
2845,1852-07-28,1952-07-28,1852.0,Invalid,atlantic ocean,nan,NaN,nan,Karen Bredesen Stræte,F,NaN,Death preceded shark involvement,False,NaN,Invalid,Norway Heritage,1852.07.28-Karen.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1852-07-28,1852-07-28,284.0,NaN,NaN
2851,1831-01-22,1831-01-22,1831.0,Invalid,australia,Tasmania,Hobart,"boat capsized, clinging to line",Robert Dudlow,M,NaN,"Drowned, no shark involvement",False,NaN,Invalid,"C. Black, GSAF; Sydney Gazette, 1/22/1831",1831.01.22.R-Dudlow.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1831-01-22,1831-01-22,215.0,NaN,NaN


In [116]:
rows_no_shark['Species '].unique()

array(['Invalid incident', 'Possibly a wobbegong',
       'Death may have been due to drowning', 'Porbeagle, 1.5 m',
       'Bronze whaler', 'Hammerhead sp.', 'Invalid', 'Wobbegong, 2m',
       'Spurdog', ' ',
       'Lesser spotted dogfish, Scyliorhinus canicula, less than 80 cm in length',
       "[4' to 5']", 'Questionable Incident', 'Unidentified species',
       '15 cm to 20 cm [6" to 8"] bite diameter just below left knee',
       'Species unidentified', '\xa0 ',
       'C. leucas tooth fragment recovered from kayak', 'Unidentified',
       'Questionable incident', '2 m hammerhead',
       'C. maculpinnis or C. limbatus', '1.8 to 2 m C. albimarginatus',
       'Considered a "Doubtful" incident', '"Blue whaler" (Galeolamna)',
       "4.9 m [16'] whaler", 'Wobbegong',
       'Dooley believed his Injury was caused by stingray (Dasyatidae family)',
       "3 m [10'] blue whaler",
       "Horn shar,k Heterodontus francisci, 1.2 m [4'] ", 'Unknown',
       'Oceanic whitetip shar,; iden

In [117]:
rows_no_shark['Species '] = np.where(rows_no_shark['Species '].isin(['Unidentified species',
       '15 cm to 20 cm [6" to 8"] bite diameter just below left knee',
       'Species unidentified', '\xa0 ',
       'C. leucas tooth fragment recovered from kayak', 'Unidentified',
       'Questionable incident', 'Unknown', 'Questionable Incident', 'Remains recovered 5 days later', '"Blue whaler" (Galeolamna)', 'Death may have been due to drowning', 'Dooley believed his Injury was caused by stingray (Dasyatidae family)', "[4' to 5']", "3 m [10'] blue whaler", 'Considered a "Doubtful" incident', "4.9 m [16'] whaler"]), np.nan, rows_no_shark['Species '])

In [118]:
rows_no_shark['Species '].unique()

array(['Invalid incident', 'Possibly a wobbegong', nan,
       'Porbeagle, 1.5 m', 'Bronze whaler', 'Hammerhead sp.', 'Invalid',
       'Wobbegong, 2m', 'Spurdog', ' ',
       'Lesser spotted dogfish, Scyliorhinus canicula, less than 80 cm in length',
       '2 m hammerhead', 'C. maculpinnis or C. limbatus',
       '1.8 to 2 m C. albimarginatus', 'Wobbegong',
       "Horn shar,k Heterodontus francisci, 1.2 m [4'] ",
       'Oceanic whitetip shar,; identified by Dr. W.C. Schoeder on photograph & Dr. L.P. L. Schultz on sketch by observer',
       'Blue pointer', 'Possibly C. leucas', "Blue pointer, 16'",
       "Blue pointer, 11' ", "Blue pointer, 12'", "15'",
       'C. macrurus captured 48 hours after attack with tissue removed from man in its gut; species identified by G.P. Whitley,  reported as C. obscurus by  R. Steel',
       'Identified as C. gangeticus by Dr. J. Fayrer'], dtype=object)

In [119]:
rows_no_shark['Species '].value_counts()

Invalid                                                                                                                                                          84
                                                                                                                                                                  5
Bronze whaler                                                                                                                                                     2
2 m hammerhead                                                                                                                                                    2
Possibly C. leucas                                                                                                                                                2
Invalid incident                                                                                                                                                  1
Horn shar,k Hete

In [120]:
nan_cols = rows_no_shark.isna().sum()

nan_cols[nan_cols>0]

Area             1
Location        10
Name            26
Age             78
Injury           2
Fatal (Y/N)      1
Time           119
Species         48
Unnamed: 22    162
Unnamed: 23    162
dtype: int64

In [121]:
# ahora aplicamos la limpieza que hemos realizado pero al dataframe con el que estamos trabajando.

In [122]:
df['Species '] = np.where(df['Species '].isin(['Unidentified species',
       '15 cm to 20 cm [6" to 8"] bite diameter just below left knee',
       'Species unidentified', '\xa0 ', 'No shark involvement',
       'C. leucas tooth fragment recovered from kayak', 'Unidentified',
       'Questionable incident', 'Unknown', 'Questionable Incident', 'Remains recovered 5 days later', '"Blue whaler" (Galeolamna)', 'Death may have been due to drowning', 'Dooley believed his Injury was caused by stingray (Dasyatidae family)', "[4' to 5']", "3 m [10'] blue whaler", 'Considered a "Doubtful" incident', "4.9 m [16'] whaler"]), np.nan, df['Species '])

In [123]:
rows_no_shark['Species '].value_counts()

Invalid                                                                                                                                                          84
                                                                                                                                                                  5
Bronze whaler                                                                                                                                                     2
2 m hammerhead                                                                                                                                                    2
Possibly C. leucas                                                                                                                                                2
Invalid incident                                                                                                                                                  1
Horn shar,k Hete

In [124]:
nan_cols = df.isna().sum()

nan_cols[nan_cols>0]

Case Number                  3
Year                         1
Area                         2
Location                    87
Name                       209
Age                        921
Injury                       4
Fatal (Y/N)                 12
Time                      1378
Species                     69
Investigator or Source       4
Case Number.1                3
Case Number.2                2
Unnamed: 22               2855
Unnamed: 23               2854
dtype: int64

In [125]:
df = df.dropna(subset=['Species '])

In [126]:
df.shape

(2786, 24)

In [127]:
df.reset_index(drop=True, inplace=True)

In [128]:
df.head(20)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018-06-25,2018-06-25,2018.0,Boat,usa,California,"Oceanside, San Diego County",paddling,Julie Wolfe,F,57.0,"No injury to occupant, outrigger canoe and pad...",False,18:00:00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-25,2018-06-25,6303.0,NaN,NaN
1,2018-06-08,2018-06-08,2018.0,Unprovoked,australia,New Wales,Arrawarra Headland,surfing,NaN,M,NaN,Minor injury to lower leg,False,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-08,2018-06-08,6300.0,NaN,NaN
2,2018-06-04,2018-06-04,2018.0,Provoked,mexico,Colima,La Ticla,free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,False,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-04,2018-06-04,6299.0,NaN,NaN
3,2018-06-03,2018-06-03,2018.0,Unprovoked,brazil,Pernambuco,"Piedade Beach, Recife",swimming,Jose Ernesto da Silva,M,18.0,FATAL,True,NaN,Tiger shark,"Diario de Pernambuco, 6/4/2018",2018.06.03.a-daSilva.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-03,2018-06-03,6297.0,NaN,NaN
4,2018-05-27,2018-05-27,2018.0,Unprovoked,usa,Florida,"Lighhouse Point Park, Ponce Inlet, Volusia County",fishing,NaN,M,52.0,Minor injury to foot. PROVOKED INCIDENT,False,NaN,"Lemon shark, 3'","K. McMurray, TrackingSharks.com",2018.05.27-Ponce.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-05-27,2018-05-27,6296.0,NaN,NaN
5,2018-05-26,2018-05-26,2018.0,Unprovoked,usa,Florida,"Cocoa Beach, Brevard County",wading,Cody High,M,15.0,Lower left leg bitten,False,17:00:00,"Bull shark, 6'","K.McMurray, TrackingSharks.com",2018.05.26.b-High.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-05-26,2018-05-26,6295.0,NaN,NaN
6,2018-05-24,2018-05-24,2018.0,Provoked,australia,Queensland,Cairns Aquarium,feeding sharks,NaN,M,32.0,Minor bite to hand by captive shark. PROVOKED ...,False,NaN,Grey reef shark,"ABC.net.au ,05/24/2018",2018.05.24-CairnsAquarium.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-05-24,2018-05-24,6293.0,NaN,NaN
7,2018-05-13,2018-05-13,2018.0,Invalid,england,Cornwall,Off Land's End,fishing,Max Berryman,M,21.0,Injured by teeth of a dead porbeagle shark he ...,False,08:15:00,Invalid incident,"K. McMurray, TrackingSharks.com",2018.05.13.a-Berryman.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-05-13,2018-05-13,6290.0,NaN,NaN
8,2018-04-28,2018-04-28,2018.0,Unprovoked,costa rica,Cocos Island,Manuelita,scuba diving,NaN,M,30.0,"No injury, shark bit scuba gear",False,10:40:00,Tiger shark,Costa Rica Star. 4/28/2018,2018.04.28.b-CostaRica.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-04-28,2018-04-28,6285.0,NaN,NaN
9,2018-04-25,2018-04-25,2018.0,Unprovoked,australia,Western Australia,"Surfers Point, Prevelly",surfing,Rob Bruce,M,60.0,"No injury, knocked off board by shark",False,09:30:00,3 m shark,"B.Myatt, GSAF",2018.04.25.a-Bruce.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-04-25,2018-04-25,6282.0,NaN,NaN


# 12. Rellenar nulos del DataFrame

In [129]:
nan_cols = df.isna().sum()

nan_cols[nan_cols>0]

Case Number                  3
Year                         1
Area                         1
Location                    83
Name                       197
Age                        897
Injury                       2
Fatal (Y/N)                 12
Time                      1335
Investigator or Source       4
Case Number.1                3
Case Number.2                2
Unnamed: 22               2786
Unnamed: 23               2785
dtype: int64

In [130]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2786 entries, 0 to 2785
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Case Number             2783 non-null   datetime64[ns]
 1   Date                    2786 non-null   datetime64[ns]
 2   Year                    2785 non-null   float64       
 3   Type                    2786 non-null   object        
 4   Country                 2786 non-null   object        
 5   Area                    2785 non-null   object        
 6   Location                2703 non-null   object        
 7   Activity                2786 non-null   object        
 8   Name                    2589 non-null   object        
 9   Sex                     2786 non-null   object        
 10  Age                     1889 non-null   float64       
 11  Injury                  2784 non-null   object        
 12  Fatal (Y/N)             2774 non-null   object  

In [131]:
df.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [132]:
df['Area'] = df['Area'].fillna('unknown')

In [133]:
df['Location'] = df['Location'].fillna('unknown')

In [134]:
df['Name'] = df['Name'].fillna('unknown')

In [135]:
df['Injury'] = df['Injury'].fillna('unknown')

In [136]:
df['Fatal (Y/N)'] = df['Fatal (Y/N)'].fillna('unknown')

In [137]:
# Quizá sería convebiente pasar el tipo de dato de esta columna a datetime y rellenar despues los nulos.

df['Time'] = df['Time'].fillna('unknown')

In [138]:
df['Age'] = df['Age'].fillna(df['Age'].median())

In [139]:
df['Case Number'] = df['Case Number'].fillna('unknown')

In [140]:
df['Case Number.1'] = df['Case Number.1'].fillna('unknown')

In [141]:
df['Case Number.2'] = df['Case Number.2'].fillna('unknown')

In [142]:
nan_cols = df.isna().sum()

nan_cols[nan_cols>0]

Year                         1
Investigator or Source       4
Unnamed: 22               2786
Unnamed: 23               2785
dtype: int64

In [143]:
df['Unnamed: 22'] = df['Unnamed: 22'].fillna('0')

In [144]:
df['Unnamed: 23'] = df['Unnamed: 23'].fillna('0')

In [145]:
df['Investigator or Source'] = df['Investigator or Source'].fillna('unknown')

In [146]:
nan_cols = df.isna().sum()

nan_cols[nan_cols>0]

Year    1
dtype: int64

In [147]:
# Analizamos que es lo que pasa con el único valor nulo de Year

In [148]:
bad_index = df['Year'][df['Year'].isna()].index

bad_index

Int64Index([107], dtype='int64')

In [149]:
df.loc[[107]]

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
107,2017-01-08 00:00:00,2017-01-08,NaN,Invalid,australia,Queensland,unknown,fishing,Kerry Daniel,M,35.0,"No attack, shark made a threat display",False,unknown,Bull shark,Liquid Vision 1/8/2017,2017.01.08.R-KerryDaniel.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017-01-08 00:00:00,2017-01-08 00:00:00,6116.0,0,0


In [150]:
# Como sabemos el año en el que se produjo porque viene especificado en la fecha vamos a modificarlo:
#df.at[i, 'columna'] = nuevo_valor
df.at[107, 'Year'] = 2017

In [151]:
nan_cols = df.isna().sum()

nan_cols[nan_cols>0]

Series([], dtype: int64)

In [152]:
df['Case Number'] = df.index
df['Case Number.1'] = df.index
df['Case Number.2'] = df.index
df['original order'] = df.index


In [153]:
df

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,0,2018-06-25,2018.0,Boat,usa,California,"Oceanside, San Diego County",paddling,Julie Wolfe,F,57.0,"No injury to occupant, outrigger canoe and pad...",False,18:00:00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,0,0,0,0,0
1,1,2018-06-08,2018.0,Unprovoked,australia,New Wales,Arrawarra Headland,surfing,unknown,M,25.0,Minor injury to lower leg,False,unknown,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1,1,1,0,0
2,2,2018-06-04,2018.0,Provoked,mexico,Colima,La Ticla,free diving,Gustavo Ramos,M,25.0,Lacerations to leg & hand shark PROVOKED INCIDENT,False,unknown,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2,2,2,0,0
3,3,2018-06-03,2018.0,Unprovoked,brazil,Pernambuco,"Piedade Beach, Recife",swimming,Jose Ernesto da Silva,M,18.0,FATAL,True,unknown,Tiger shark,"Diario de Pernambuco, 6/4/2018",2018.06.03.a-daSilva.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,3,3,3,0,0
4,4,2018-05-27,2018.0,Unprovoked,usa,Florida,"Lighhouse Point Park, Ponce Inlet, Volusia County",fishing,unknown,M,52.0,Minor injury to foot. PROVOKED INCIDENT,False,unknown,"Lemon shark, 3'","K. McMurray, TrackingSharks.com",2018.05.27-Ponce.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,4,4,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2781,2781,1834-07-15,1834.0,Unprovoked,french polynesia,Society Islands,Tahiti,swimming,Kaugatava Orurutm,F,25.0,FATAL,True,unknown,Reported to involve a hammerhead shark,"Republican Banner, 7/15/1834",1834.07.15.R-Tahiti.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2781,2781,2781,0,0
2782,2782,1831-01-22,1831.0,Invalid,australia,Tasmania,Hobart,"boat capsized, clinging to line",Robert Dudlow,M,25.0,"Drowned, no shark involvement",False,unknown,Invalid,"C. Black, GSAF; Sydney Gazette, 1/22/1831",1831.01.22.R-Dudlow.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2782,2782,2782,0,0
2783,2783,1819-07-08,1819.0,Invalid,spain,nan,Cadiz,nan,unknown,M,25.0,No injury / No attack,False,unknown,Invalid,"C. Moore, GSAF",1819.07.08.R-Cadiz.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2783,2783,2783,0,0
2784,2784,1801-12-18,1801.0,Provoked,nan,nan,unknown,wading,Stephen Pettigew,nan,25.0,"FATAL, PROVOKED INCIDENT",True,unknown,12' shark,"The Evening Post, 12/18/1801",1801.12.18.R-Pettigrew.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2784,2784,2784,0,0


In [154]:
df.duplicated().any()

False

## Optimización de la memoria cambiando el tipo de dato

In [155]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2786 entries, 0 to 2785
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Case Number             2786 non-null   int64         
 1   Date                    2786 non-null   datetime64[ns]
 2   Year                    2786 non-null   float64       
 3   Type                    2786 non-null   object        
 4   Country                 2786 non-null   object        
 5   Area                    2786 non-null   object        
 6   Location                2786 non-null   object        
 7   Activity                2786 non-null   object        
 8   Name                    2786 non-null   object        
 9   Sex                     2786 non-null   object        
 10  Age                     2786 non-null   float64       
 11  Injury                  2786 non-null   object        
 12  Fatal (Y/N)             2786 non-null   object  

In [156]:
for c in df.select_dtypes(include='object'):
    
    df[c] = df[c].astype('category')   # cambia el tipo a categoria
    
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2786 entries, 0 to 2785
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Case Number             2786 non-null   int64         
 1   Date                    2786 non-null   datetime64[ns]
 2   Year                    2786 non-null   float64       
 3   Type                    2786 non-null   category      
 4   Country                 2786 non-null   category      
 5   Area                    2786 non-null   category      
 6   Location                2786 non-null   category      
 7   Activity                2786 non-null   category      
 8   Name                    2786 non-null   category      
 9   Sex                     2786 non-null   category      
 10  Age                     2786 non-null   float64       
 11  Injury                  2786 non-null   category      
 12  Fatal (Y/N)             2786 non-null   category

In [157]:
for c in df.select_dtypes('integer'):
    
    df[c] = pd.to_numeric(df[c], downcast='integer')
    
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2786 entries, 0 to 2785
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Case Number             2786 non-null   int16         
 1   Date                    2786 non-null   datetime64[ns]
 2   Year                    2786 non-null   float64       
 3   Type                    2786 non-null   category      
 4   Country                 2786 non-null   category      
 5   Area                    2786 non-null   category      
 6   Location                2786 non-null   category      
 7   Activity                2786 non-null   category      
 8   Name                    2786 non-null   category      
 9   Sex                     2786 non-null   category      
 10  Age                     2786 non-null   float64       
 11  Injury                  2786 non-null   category      
 12  Fatal (Y/N)             2786 non-null   category

In [158]:
for c in df.select_dtypes('float'):
    
    df[c] = pd.to_numeric(df[c], downcast='float')
    
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2786 entries, 0 to 2785
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Case Number             2786 non-null   int16         
 1   Date                    2786 non-null   datetime64[ns]
 2   Year                    2786 non-null   float32       
 3   Type                    2786 non-null   category      
 4   Country                 2786 non-null   category      
 5   Area                    2786 non-null   category      
 6   Location                2786 non-null   category      
 7   Activity                2786 non-null   category      
 8   Name                    2786 non-null   category      
 9   Sex                     2786 non-null   category      
 10  Age                     2786 non-null   float32       
 11  Injury                  2786 non-null   category      
 12  Fatal (Y/N)             2786 non-null   category

In [159]:
for c in df.select_dtypes('datetime64[ns]'):
    
    df[c]=df[c].astype('datetime64[s]')
    
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2786 entries, 0 to 2785
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Case Number             2786 non-null   int16         
 1   Date                    2786 non-null   datetime64[ns]
 2   Year                    2786 non-null   float32       
 3   Type                    2786 non-null   category      
 4   Country                 2786 non-null   category      
 5   Area                    2786 non-null   category      
 6   Location                2786 non-null   category      
 7   Activity                2786 non-null   category      
 8   Name                    2786 non-null   category      
 9   Sex                     2786 non-null   category      
 10  Age                     2786 non-null   float32       
 11  Injury                  2786 non-null   category      
 12  Fatal (Y/N)             2786 non-null   category

In [160]:
df.to_csv(r'..\data\shark_attack_clean.csv', index=False)